# Cell clustering notebook

NOTE: this notebook should be run after `example_pixel_clustering.ipynb`

In [1]:
# import required packages
from datetime import datetime as dt
import os
import subprocess

import feather
import json
from matplotlib import rc_file_defaults
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import xarray as xr

from ark.analysis import visualize
from ark.phenotyping import som_utils
from ark.utils import data_utils, io_utils, load_utils, plot_utils
from ark.utils.metacluster_remap_gui import MetaClusterData, MetaClusterGui, metaclusterdata_from_files

## 1: Load parameters for cell clustering (computed by `example_pixel_clustering.ipynb`)

`cell_clustering_params_name` should be in the form `{pixel_cluster_prefix}_cell_clustering_params.json` contained in `{pixel_cluster_prefix}_pixel_output_dir`. Make sure to set `base_dir` and `pixel_output_dir` to the same value used in `example_pixel_clustering.ipynb`.

NOTE: `{pixel_cluster_prefix}` is set in `example_pixel_clustering.ipynb`. If you did not explicity set a `{pixel_cluster_prefix}` in `example_pixel_clustering.ipynb`, the prefix defaults to the timestamp of the run. Please check the run directory (`base_dir` as set in `example_pixel_clustering.ipynb`) to see the prefix that was used.

In [4]:
base_dir = "../data/external/TONIC_Cohort"

# define the name of the folder containing the pixel cluster data
pixel_prefix = '20220707_full_cohort'
pixel_output_dir = pixel_prefix + '_pixel_output_dir'

# define the name of the cell clustering params file
cell_clustering_params_name = pixel_prefix + '_cell_clustering_params.json'

The following params are loaded:

* `fovs`: the subset of fovs used for pixel clustering
* `channels`: the subset of channels used for pixel clustering
* `segmentation_dir`: the path to the directory containing your segmented images (generated from `Segment_Image_Data.ipynb`)
* `seg_suffix`: the suffix plus the file extension of the segmented images for each FOV
* `pixel_data_dir`: the name of the directory containing pixel data with the pixel SOM and consensus cluster assignments
* `pc_chan_avg_som_cluster_name`: the name of the file containing the average channel expression per pixel SOM cluster, used for the visualization of weighted channel average per cell
* `pc_chan_avg_meta_cluster_name`: the name of the file containing the average channel expression per pixel meta cluster, used for the visualization of weighted channel average per cell

Additionally, define the following param:

* `cell_table_path`: path to the cell table created by `Segment_Image_Data.ipynb`, should be placed in `segmentation_dir` by default. You can use either the normalized or arcsinh versions (the only columns from the cell table used are `fov`, `label`, and `cell_size`, so the clustering pipeline supports both).

In [14]:
# load the params
with open(os.path.join(base_dir, pixel_output_dir, cell_clustering_params_name)) as fh:
    cell_clustering_params = json.load(fh)
    
# assign the params to variables
fovs = cell_clustering_params['fovs']
channels = cell_clustering_params['channels']
segmentation_dir = '../data/external/TONIC_Cohort/segmentation_data/single_cell_output'
seg_suffix = cell_clustering_params['seg_suffix']
pixel_data_dir = cell_clustering_params['pixel_data_dir']
pc_chan_avg_som_cluster_name = cell_clustering_params['pc_chan_avg_som_cluster_name']
pc_chan_avg_meta_cluster_name = cell_clustering_params['pc_chan_avg_meta_cluster_name']

# define the cell table path
cell_table_path = os.path.join(segmentation_dir, 'combined_cell_table_normalized.csv')

## 2: Cell clustering

### 2.1: train cell SOM

Set a prefix to be applied to all data directories/files created by pixel and cell clustering. If the prefix is not set, a default of the datetime at the start of the run is used.

In [6]:
# explicitly set cell_cluster_prefix to override datetime default
cell_cluster_prefix = '20220715_full_cohort'

if cell_cluster_prefix is None:
    cell_cluster_prefix = dt.now().strftime('%Y-%m-%dT%H:%M:%S')

The following folders/files will be created with names prefixed by `cell_cluster_prefix`:

* `cell_output_dir`: the name of the folder to store the cell clustering directories/files
* `cell_data_name`: file to store the cell data with the eventual SOM and consensus cluster assignments
* `cell_weights_name`: file name to place the cell SOM weights
* `cluster_counts_name`: file name to store the counts of each pixel SOM/meta clusters per cell
* `cluster_counts_norm_name`: same as above, except with each value normalized by the respective cell's size
* `weighted_cell_channel_name`: file name to store the weighted cell channel expression for each cell. Refer to <a href=https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.phenotyping.html#ark.phenotyping.som_utils.compute_p2c_weighted_channel_avg>cell channel weighting docs</a> for how the weighting is computed.
* `cell_clust_to_meta_name`: file name to store the mapping between cell SOM clusters and cell meta clusters
* `cell_som_cluster_count_avgs_name`: file name to store the average number of pixel SOM/meta clusters per cell SOM cluster
* `cell_meta_cluster_count_avgs_name`: same as above except for cell meta clusters
* `cell_som_cluster_channel_avg_name`: file name to store the average weighted channel expression per cell SOM cluster
* `cell_meta_cluster_channel_avg_name`: same as above except for cell meta clusters
* `cell_meta_cluster_remap_name`: for the meta cluster remapping process, the file to store the new SOM to meta mappings

In [7]:
# define the base output cell folder
cell_output_dir = '%s_cell_output_dir' % cell_cluster_prefix
if not os.path.exists(os.path.join(base_dir, cell_output_dir)):
    os.mkdir(os.path.join(base_dir, cell_output_dir))
    
# define the paths to cell clustering files, explicitly set the variables to use custom names
cell_data_name = os.path.join(cell_output_dir, '%s_cell_mat.feather' % cell_cluster_prefix)
cell_weights_name = os.path.join(cell_output_dir, '%s_cell_weights.feather' % cell_cluster_prefix)
cluster_counts_name = os.path.join(cell_output_dir, '%s_cluster_counts.feather' % cell_cluster_prefix)
cluster_counts_norm_name = os.path.join(cell_output_dir, '%s_cluster_counts_norm.feather' % cell_cluster_prefix)
weighted_cell_channel_name = os.path.join(cell_output_dir, '%s_weighted_cell_channel.csv' % cell_cluster_prefix)
cell_clust_to_meta_name = os.path.join(cell_output_dir, '%s_cell_clust_to_meta.feather' % cell_cluster_prefix)
cell_som_cluster_count_avgs_name = os.path.join(cell_output_dir, '%s_cell_som_cluster_count_avgs.csv' % cell_cluster_prefix)
cell_meta_cluster_count_avgs_name = os.path.join(cell_output_dir, '%s_cell_meta_cluster_count_avgs.csv' % cell_cluster_prefix)
cell_som_cluster_channel_avg_name = os.path.join(cell_output_dir, '%s_cell_som_cluster_channel_avg.csv' % cell_cluster_prefix)
cell_meta_cluster_channel_avg_name = os.path.join(cell_output_dir, '%s_cell_meta_cluster_channel_avg.csv' % cell_cluster_prefix)
cell_meta_cluster_remap_name = os.path.join(cell_output_dir, '%s_cell_meta_cluster_mapping.csv' % cell_cluster_prefix)

Additionally, define the following pixel cluster aggregation variable:

* `pixel_cluster_col`: whether to use pixel SOM or meta cluster counts for training/aggregating. Set to `'pixel_som_cluster'` or `'pixel_meta_cluster_rename'` respectively. Note that if you did not explicitly rename your pixel meta clusters in `example_pixel_clustering.ipynb`, the default numeric names will be used.

In [8]:
# define the type of pixel cluster to aggregate on
pixel_cluster_col = 'pixel_meta_cluster_rename'

# depending on which pixel_cluster_col is selected, choose the pixel channel average table accordingly
if pixel_cluster_col == 'pixel_som_cluster':
    pc_chan_avg_name = pc_chan_avg_som_cluster_name
elif pixel_cluster_col == 'pixel_meta_cluster_rename':
    pc_chan_avg_name = pc_chan_avg_meta_cluster_name

Train the cell SOM on the normalized number of pixel SOM/meta clusters per cell (the data stored in `cluster_counts_norm_name`).  Training is done using the `FlowSOM` algorithm. Note that each of the pixel SOM/meta cluster columns are normalized by their 99.9% value prior to training.

Additionally, this function also computes the weighted cell channel table (the data stored in `weighted_cell_channel_name`). This data will be needed to compute the weighted average channel expression per cell cluster (the data stored in `cell_som_cluster_channel_avg_name` and `cell_meta_cluster_channel_avg_name`). See documentation of `compute_p2c_weighted_channel_avg` for how weighted cell channel average is computed: <a href=https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.phenotyping.html#ark.phenotyping.som_utils.compute_p2c_weighted_channel_avg>cell channel weighting docs</a>.

For a full set of parameters you can customize for `train_cell_som`, please consult: <a href=https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.phenotyping.html#ark.phenotyping.som_utils.train_cell_som>cell training docs</a>

In [7]:
# create the cell-level SOM weights
som_utils.train_cell_som(
    fovs,
    channels,
    base_dir,
    pixel_data_dir=pixel_data_dir + '_remapped',
    cell_table_path=cell_table_path,
    cluster_counts_name=cluster_counts_name,
    cluster_counts_norm_name=cluster_counts_norm_name,
    pixel_cluster_col=pixel_cluster_col,
    pc_chan_avg_name=pc_chan_avg_name,
    weights_name=cell_weights_name,
    weighted_cell_channel_name=weighted_cell_channel_name,
    num_passes=1,
    xdim=17,
    ydim=17
)

Counting the number of pixel SOM/meta cluster counts for each fov/cell pair
Some features are not enabled in this build of Arrow. Run `arrow_info()` for more information.

Attaching package: ‘arrow’

The following object is masked from ‘package:utils’:

timestamp

Loading required package: igraph

Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

decompose, spectrum

The following object is masked from ‘package:base’:

union

Thanks for using FlowSOM. From version 2.1.4 on, the scale
parameter in the FlowSOM function defaults to FALSE
[1] "Reading the cluster counts data for SOM training"
[1] "Run the SOM training"
[1] "Save trained weights"
Computing the weighted channel expression per cell


### 2.2: assign cell SOM clusters

Use the weights learned from `train_cell_som` to assign cell clusters to the cell dataset. Note that this is done on the normalized pixel SOM/meta cluster counts table. As with `train_pixel_som`, each of the columns are normalized by their 99.9% value prior to assigning a cell SOM cluster label.

This function also computes the average number of pixel SOM/meta clusters per cell SOM cluster as well as the number of cells in each cell SOM cluster (the data placed in `cell_som_cluster_count_avgs_name`). This is needed for cell consensus clustering.

In [8]:
# use cell SOM weights to assign cell clusters
som_utils.cluster_cells(
    base_dir,
    cluster_counts_norm_name=cluster_counts_norm_name,
    weights_name=cell_weights_name,
    cell_data_name=cell_data_name,
    pixel_cluster_col_prefix=pixel_cluster_col,
    cell_som_cluster_count_avgs_name=cell_som_cluster_count_avgs_name
)

Some features are not enabled in this build of Arrow. Run `arrow_info()` for more information.

Attaching package: ‘arrow’

The following object is masked from ‘package:utils’:

timestamp

Loading required package: igraph

Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

decompose, spectrum

The following object is masked from ‘package:base’:

union

Thanks for using FlowSOM. From version 2.1.4 on, the scale
parameter in the FlowSOM function defaults to FALSE
[1] "Reading the cluster counts data"
[1] "Reading the weights matrix"
[1] "Perform 99.9% normalization"
[1] "Mapping cluster labels"
[1] "Writing clustered data"
Computing the average number of pixel SOM/meta cluster counts per cell SOM cluster


### 2.3: run cell consensus clustering

With the SOM cluster labels assigned to the cell data, assign consensus cluster labels. The consensus clusters are trained on the average number of pixel SOM/meta clusters across all cell SOM clusters (the data stored in `cell_som_cluster_count_avgs_name`). These values are z-scored and capped at the value specified in the `cap` argument prior to training: this helps improve the meta clustering process.

After consensus clustering, the following are also computed:

* The average number of pixel SOM/meta clusters across all cell meta clusters, and the number of cells per meta cluster (the data placed in `cell_meta_cluster_count_avgs_name`)
* The meta cluster mapping for each cell SOM cluster in `cell_som_cluster_count_avgs_name` (data is resaved, same data except with an associated meta cluster column)
* The weighted channel average across all cell clusters (the data placed in `cell_som_cluster_channel_avgs_name` and `cell_meta_cluster_channel_avgs_name`). This will be done for both `'cell_som_cluster'` and `'cell_meta_cluster'`.

For a full set of parameters you can customize for `cell_consensus_cluster`, please consult: <a href=https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.phenotyping.html#ark.phenotyping.som_utils.cell_consensus_cluster>cell consensus clustering docs</a>

* `max_k`: the number of consensus clusters desired
* `cap`: used to clip z-scored values prior to consensus clustering (in the range `[-cap, cap]`)

In [9]:
max_k = 30
cap = 3

# run hierarchical clustering based on cell SOM cluster assignments
som_utils.cell_consensus_cluster(
    fovs=fovs,
    channels=channels,
    base_dir=base_dir,
    pixel_cluster_col=pixel_cluster_col,
    max_k=max_k,
    cap=cap,
    cell_data_name=cell_data_name,
    cell_som_cluster_count_avgs_name=cell_som_cluster_count_avgs_name,
    cell_meta_cluster_count_avgs_name=cell_meta_cluster_count_avgs_name,
    weighted_cell_channel_name=weighted_cell_channel_name,
    cell_som_cluster_channel_avg_name=cell_som_cluster_channel_avg_name,
    cell_meta_cluster_channel_avg_name=cell_meta_cluster_channel_avg_name,
    clust_to_meta_name=cell_clust_to_meta_name
)

Some features are not enabled in this build of Arrow. Run `arrow_info()` for more information.

Attaching package: ‘arrow’

The following object is masked from ‘package:utils’:

timestamp

[1] "Reading cluster averaged data"
[1] "Scaling data"
[1] "Running consensus clustering"
[1] "Writing consensus clustering"
[1] "Writing SOM to meta cluster mapping table"
Compute the average number of pixel SOM/meta cluster counts per cell meta cluster
Mapping meta cluster values onto average number of pixel SOM/meta cluster countsacross cell SOM clusters
Compute average weighted channel expression across cell SOM clusters
Mapping meta cluster values onto average weighted channel expressionacross cell SOM clusters
Compute average weighted channel expression across cell meta clusters


## 3: visualize results

### 3.1: use the interactive reclustering results to relabel cell meta clusters

The visualization shows the z-scored average pixel cluster count expression per cell SOM and meta cluster. The heatmaps are faceted by cell SOM clusters on the left and cell meta clusters on the right.

## Usage

### Quickstart
- **Select**: Left Click
- **Remap**: **New metacluster button** or Right Click
- **Edit Metacluster Name**: Textbox at bottom right of the heatmaps.

### Selection and Remapping details
- To select a SOM cluster, click on its respective position in the **selected** bar. Click on it again to deselect.
- To select a meta cluster, click on its corresponding color in the **metacluster** bar. Click on it again to deselect.
- To remap the selected clusters, click the **New metacluster** button (alternatively, right click anywhere). Note that remapping an entire metacluster deletes it.
- To clear the selected SOM/meta clusters, use the **Clear Selection** button.
- **After remapping a meta cluster, make sure to deselect the newly created one to prevent unwanted combinations.**

### Other features and notes
- You will likely need to zoom out to see the entire visualization. To toggle Zoom, use Ctrl -/Ctrl + on Windows or ⌘ +/⌘ - on Mac.
- The bars at the top show the number of cells in each SOM cluster.
- The text box at the bottom right allows you to rename a particular meta cluster. This can be useful as remapping may cause inconsistent numbering.
- Adjust the z-score limit using the slider on the bottom left to adjust your dynamic range.
- When meta clusters are combined or a meta cluster is renamed, the change is immediately saved to `cell_meta_cluster_remap_name`.
- You won't be able to advance until you've clicked `New metacluster` or renamed a meta cluster at least once. If you do not want to make changes, just click `New metacluster` to trigger a save before continuing.

In [9]:
%matplotlib widget
rc_file_defaults()
plt.ion()

cell_mcd = metaclusterdata_from_files(
    os.path.join(base_dir, cell_som_cluster_count_avgs_name),
    cluster_type='cell',
    prefix_trim='pixel_meta_cluster_rename_'
)
cell_mcd.output_mapping_filename = os.path.join(base_dir, cell_meta_cluster_remap_name)
cell_mcg = MetaClusterGui(cell_mcd, width=17)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Relabel the cell meta clusters using the mapping.

In [10]:
som_utils.apply_cell_meta_cluster_remapping(
    fovs,
    channels,
    base_dir,
    cell_data_name,
    cell_meta_cluster_remap_name,
    pixel_cluster_col,
    cell_som_cluster_count_avgs_name,
    cell_meta_cluster_count_avgs_name,
    weighted_cell_channel_name,
    cell_som_cluster_channel_avg_name,
    cell_meta_cluster_channel_avg_name
)

Using re-mapping scheme to re-label cell meta clusters
Re-compute pixel SOM/meta cluster count per cell meta cluster
Re-compute average weighted channel expression across cell meta clusters
Re-assigning meta cluster column in cell SOM cluster average pixel cluster counts data
Re-assigning meta cluster column in cell SOM cluster average weighted channel data


Generate the color scheme returned by the interactive reclustering process. This will be for visualizing the weighted channel average heatmaps and the cell cluster overlay.

In [10]:
raw_cmap, renamed_cmap = som_utils.generate_meta_cluster_colormap_dict(
    cell_mcd.output_mapping_filename,
    cell_mcg.im_cl.cmap
)

### 3.2: weighted cell SOM cluster average heatmap over channels (z-scored)

In [11]:
som_utils.generate_weighted_channel_avg_heatmap(
    os.path.join(base_dir, cell_som_cluster_channel_avg_name),
    'cell_som_cluster',
    channels,
    raw_cmap,
    renamed_cmap
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

### 3.3: weighted cell meta cluster average heatmap over channels (z-scored)

In [12]:
som_utils.generate_weighted_channel_avg_heatmap(
    os.path.join(base_dir, cell_meta_cluster_channel_avg_name),
    'cell_meta_cluster_rename',
    channels,
    raw_cmap,
    renamed_cmap
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

### 3.4: cell cluster overlay (cell meta clusters only)

In [16]:
import random
import copy
test_fovs = copy.deepcopy(fovs)
random.shuffle(test_fovs)
test_fovs[:20]

# select fovs to display
cell_fovs = test_fovs[:20]
cell_fovs

['TONIC_TMA8_R4C6',
 'TONIC_TMA21_R3C3',
 'TONIC_TMA3_R9C6',
 'TONIC_TMA24_R6C6',
 'TONIC_TMA14_R5C1',
 'TONIC_TMA3_R1C6',
 'TONIC_TMA12_R3C5',
 'TONIC_TMA5_R6C5',
 'TONIC_TMA12_R8C5',
 'TONIC_TMA10_R10C2',
 'TONIC_TMA7_R4C4',
 'TONIC_TMA7_R12C2',
 'TONIC_TMA18_R1C5',
 'TONIC_TMA19_R8C2',
 'TONIC_TMA6_R11C5',
 'TONIC_TMA22_R2C5',
 'TONIC_TMA2_R7C3',
 'TONIC_TMA12_R7C4',
 'TONIC_TMA18_R5C1',
 'TONIC_TMA16_R9C4']

In [19]:
cell_fovs = ['TONIC_TMA23_R6C1',
 'TONIC_TMA15_R11C2',
 'TONIC_TMA23_R9C4',
 'TONIC_TMA6_R7C4',
 'TONIC_TMA23_R6C3',
 'TONIC_TMA18_R2C2',
 'TONIC_TMA19_R6C6',
 'TONIC_TMA18_R9C1',
 'TONIC_TMA4_R1C2',
 'TONIC_TMA23_R1C2',
 'TONIC_TMA10_R4C1',
 'TONIC_TMA10_R3C6',
 'TONIC_TMA1_R8C1',
 'TONIC_TMA14_R3C4',
 'TONIC_TMA13_R5C1',
 'TONIC_TMA7_R7C3',
 'TONIC_TMA12_R3C4',
 'TONIC_TMA6_R5C2']

In [20]:
# generate the cell cluster masks for each fov in cell_fovs
cell_cluster_masks = data_utils.generate_cell_cluster_mask(
    cell_fovs,
    base_dir,
    '../data/external/segmentation_data/deepcell_output/',
    cell_data_name=cell_data_name,
    cell_cluster_col='cell_meta_cluster',
    seg_suffix=seg_suffix
)

/usr/local/lib/python3.6/site-packages/ark/utils/load_utils.py:302: UserWarning: The loaded float32 images were forcefully overwritten with the supplied integer dtype int16
  warnings.warn(f"The loaded {data_dtype} images were forcefully "


* `save_cell_masks`: replace with `True` if you want to save, files will be written as `{fov_name}_cell_mask.tiff` in `base_dir`

In [21]:
save_cell_masks = True

if save_cell_masks:
    data_utils.save_fov_images(
        cell_fovs,
        os.path.join(base_dir, cell_output_dir),
        cell_cluster_masks,
        sub_dir='cell_masks',
        name_suffix='_cell_mask'
    )

In [24]:
# mantis creation
create_mantis_project(mantis_project_path=os.path.join(base_dir, cell_output_dir, 'mantis'),
                      img_data_path= os.path.join(base_dir, 'image_data/samples'),
                      mask_output_dir=os.path.join(base_dir, cell_output_dir, 'cell_masks'),
                      mask_suffix='_cell_mask',
                      mapping_path = os.path.join(base_dir, cell_output_dir, cell_cluster_prefix + '_cell_meta_cluster_mapping.csv'),
                     seg_dir= '../data/external/segmentation_data/deepcell_output/', img_sub_folder='')


In [ ]:
plot_utils.plot_pixel_cell_cluster_overlay(
    cell_cluster_masks,
    cell_fovs,
    os.path.join(base_dir, cell_meta_cluster_remap_name),
    metacluster_colors=raw_cmap
)

### 3.5: save consensus cluster labels to cell table

"The cell table with consensus cluster labels is saved to `{cell_table_path}_cell_labels.csv`"

In [15]:
som_utils.add_consensus_labels_cell_table(
    base_
    dir, cell_table_path, cell_data_name
)

In [12]:
cell_data_name

'20220715_full_cohort_cell_output_dir/20220715_full_cohort_cell_mat.feather'

In [23]:
import shutil
def create_mantis_project(mantis_project_path, img_data_path, mask_output_dir, mask_suffix, mapping_path, 
                          seg_dir, img_sub_folder='normalized'):
    
    if not os.path.exists(mantis_project_path):
        os.makedirs(mantis_project_path)
        
    # create key from cluster number to cluster name
    map_df = pd.read_csv(mapping_path)
    map_df = map_df.loc[:, ['metacluster', 'mc_name']]

    # remove duplicates from df
    map_df = map_df.drop_duplicates()
    map_df = map_df.sort_values(by=['metacluster'])
    
    # rename for mantis names
    map_df = map_df.rename({'metacluster': 'region_id', 'mn_name': 'region_name'}, axis=1)
    
    # get names of fovs with masks
    mask_names = io_utils.list_files(mask_output_dir, mask_suffix)
    fov_names = io_utils.extract_delimited_names(mask_names, delimiter=mask_suffix)
    
    # create a folder with image data, pixel masks, and segmentation mask
    for idx, val in enumerate(fov_names):
        
        # set up paths
        img_source_dir = os.path.join(img_data_path, val, img_sub_folder)
        output_dir = os.path.join(mantis_project_path, val)
        
        # copy image data if not already copied in from previous round of clustering
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

            # copy all channels into new folder
            chans = io_utils.list_files(img_source_dir, '.tiff')
            for chan in chans:
                shutil.copy(os.path.join(img_source_dir, chan), os.path.join(output_dir, chan))

        # copy mask into new folder
        mask_name = mask_names[idx]
        shutil.copy(os.path.join(mask_output_dir, mask_name), os.path.join(output_dir, 'population{}.tiff'.format(mask_suffix)))
        
        # copy segmentations into directory
        seg_name = val + '_feature_0.tif'
        shutil.copy(os.path.join(seg_dir, seg_name), os.path.join(output_dir, 'cell_segmentation.tiff'))
        
        # copy mapping into directory
        map_df.to_csv(os.path.join(output_dir, 'population{}.csv'.format(mask_suffix)), index=False)